# user interface

> UI Prototypes and deployment

In [ ]:
#| default_exp app_gradio

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sbavery/pseudometer/blob/main/nbs/02_user_interface.ipynb)

In [ ]:
#| hide
## Google Colab / Installs
#!pip install gradio

In [ ]:
#| export
#| hide
import warnings
warnings.filterwarnings('ignore')
from fastai.text.all import *
from pseudometer.data import *
import gradio as gr

## Gradio Prototype

In [ ]:
#| hide
#| export
categories = ('pseudoscience','science')
k = 30
min_words = 20
max_words = 450
ignore_text = ['the', 'of', 'to', 'and', 'a', 'in', 'it', 'that', 'for', 'on'] 
ignore_common = ignore_text
learn = load_learner('models/2022.12.01 Model v1 88pct', cpu=False)

def predict(url):
    page = get_page_all(url, k, max_words, ignore_text, ignore_common)
    length = len(page.cleaned_text)
    if  length < min_words:
        return "ERROR: Returned "+str(length)+" words"
    else:
        text = ' '.join(page.cleaned_text)
        with learn.no_bar(), learn.no_logging():
            pred,idx,probs = learn.predict(text)
        return dict(zip(categories, map(float,probs)))

In [ ]:
predict('https://www.si.edu/explore/science')

{'pseudoscience': 0.0023253117688000202, 'science': 0.9976746439933777}

In [ ]:
#| export
text = gr.inputs.Textbox(1)
label = gr.outputs.Label()
examples = ['https://www.theskepticsguide.org/about','https://www.foxnews.com/opinion']

intf = gr.Interface(fn=predict, inputs=text, outputs=label, examples=examples)

In [ ]:
intf.launch(inline=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
